<a href="https://colab.research.google.com/github/hitanshu5/Crop-Yield-Prediction-System/blob/main/Crop_Weighted_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy==0.1.12

  Using cached pgmpy-0.1.12-py3-none-any.whl.metadata (6.3 kB)
Using cached pgmpy-0.1.12-py3-none-any.whl (322 kB)
  Attempting uninstall: pgmpy
    Found existing installation: pgmpy 1.0.0
    Uninstalling pgmpy-1.0.0:
      Successfully uninstalled pgmpy-1.0.0


In [ ]:
!pip install --upgrade pgmpy

  Using cached pgmpy-1.0.0-py3-none-any.whl.metadata (9.4 kB)
Using cached pgmpy-1.0.0-py3-none-any.whl (2.0 MB)
  Attempting uninstall: pgmpy
    Found existing installation: pgmpy 0.1.12
    Uninstalling pgmpy-0.1.12:
      Successfully uninstalled pgmpy-0.1.12


In [ ]:
!pip install tensorflow-gpu==2.12.0

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, LabelEncoder,StandardScaler

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, Conv1D, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
df = pd.read_csv('/content/final_df.csv')
df.head()

,District,Crop,Year,Season,Area,Temperature,Rainfall,Yield
0,1,2,1997,2,40800.0,23.998425,112.450,0.455882
1,1,4,1997,2,900.0,23.998425,112.450,1.222222
2,1,4,1997,3,598400.0,21.448471,9.375,0.362634
3,1,5,1997,2,4400.0,23.998425,112.450,1.068182
4,1,5,1997,3,6200.0,21.448471,9.375,1.467742


## CNN-LSTM model

In [ ]:
df_tsa = df.copy()

if not pd.api.types.is_datetime64_any_dtype(df_tsa['Year']):
    df_tsa['Year'] = pd.to_datetime(df_tsa['Year'].astype(str), format='%Y').dt.year
else:
    df_tsa['Year'] = df_tsa['Year'].dt.year

tsa_features = ['District', 'Area', 'Year', 'Season', 'Temperature', 'Rainfall', 'Crop']
X_tsa = df_tsa[tsa_features].values
y_tsa = df_tsa['Yield'].values.reshape(-1, 1)

scaler_tsa = StandardScaler()
X_tsa_scaled = scaler_tsa.fit_transform(X_tsa)
scaler_y = StandardScaler()
y_tsa_scaled = scaler_y.fit_transform(y_tsa)

time_steps = 3
n_features = X_tsa_scaled.shape[1]

X_tsa_seq = np.array([X_tsa_scaled[i-time_steps:i] for i in range(time_steps, len(X_tsa_scaled))])
y_tsa_seq = y_tsa_scaled[time_steps:]

X_train, X_test, y_train, y_test = train_test_split(X_tsa_seq, y_tsa_seq, test_size=0.25, random_state=42)

def build_optimized_cnn_lstm_model(time_steps, n_features):
    model = Sequential()

    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same', input_shape=(time_steps, n_features)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.2, activation='tanh')))
    model.add(Dropout(0.3))

    model.add(LSTM(64, activation='tanh', return_sequences=False, recurrent_dropout=0.2))
    model.add(Dropout(0.3))

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))

    lr_schedule = ExponentialDecay(initial_learning_rate=0.001, decay_steps=1000, decay_rate=0.9)
    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss='mse')

    return model

model_tsa = build_optimized_cnn_lstm_model(time_steps, n_features)
model_tsa.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

history = model_tsa.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test),
                        callbacks=[early_stopping], verbose=1)

y_pred_scaled = model_tsa.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_actual = scaler_y.inverse_transform(y_test)

mae = mean_absolute_error(y_test_actual, y_pred)
mse = mean_squared_error(y_test_actual, y_pred)
rmse = np.sqrt(mse)

print("\n🔹 Model Evaluation Metrics:")
print(f"✔️ Mean Absolute Error (MAE): {mae:.4f}")
print(f"✔️ Mean Squared Error (MSE): {mse:.4f}")
print(f"✔️ Root Mean Squared Error (RMSE): {rmse:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 3, 128)              │           2,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 3, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 3, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 3, 64)               │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 3, 64)               │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 3, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 3, 256)              │         197,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 3, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 314,305 (1.20 MB)

 Trainable params: 313,921 (1.20 MB)

 Non-trainable params: 384 (1.50 KB)

Epoch 1/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - loss: 0.9390 - val_loss: 0.7878
Epoch 2/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - loss: 0.8313 - val_loss: 0.7351
Epoch 3/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - loss: 0.7769 - val_loss: 0.7143
Epoch 4/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 14s 42ms/step - loss: 0.7530 - val_loss: 0.6879
Epoch 5/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 20s 42ms/step - loss: 0.7316 - val_loss: 0.6788
Epoch 6/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 21s 42ms/step - loss: 0.7183 - val_loss: 0.6708
Epoch 7/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 21s 42ms/step - loss: 0.7375 - val_loss: 0.6704
Epoch 8/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - loss: 0.6852 - val_loss: 0.6678
Epoch 9/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 21s 42ms/step - loss: 0.6975 - val_loss: 0.6557
Epoch 10/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - loss: 0.6741 - val_loss: 0.6574
Epoch 11/150
326/326 ━━━━━━━━━━━━━━━━━━━━ 14s 42ms/step - loss: 0.6830 - val_loss: 0.6459
Epoch 12/150
326/32

In [ ]:
cnn_lstm_yields = []

# Iterate through each row of the DataFrame for prediction
for index, row in df.iterrows():
    # Extract values from the DataFrame.
    # Note: The order in the CSV is:
    # District, Crop, Year, Season, Area, Temperature, Rainfall, Yield
    # But our model expects: District, Area, Year, Season, Temperature, Rainfall, Crop
    district = row["District"]
    area = row["Area"]
    year = row["Year"]
    season = row["Season"]
    temp = row["Temperature"]
    rainfall = row["Rainfall"]
    crop = row["Crop"]

    # Construct the input array in the required order.
    user_input_cnn_lstm = np.array([[district, area, year, season, temp, rainfall, crop]])

    # Scale the input features using your predefined scaler (scaler_tsa)
    user_input_scaled = scaler_tsa.transform(user_input_cnn_lstm)
    # Reshape the input to match the CNN-LSTM expected shape: (batch_size, sequence_length, features)
    user_input_reshaped = user_input_scaled.reshape(1, 1, user_input_scaled.shape[1])

    # Predict yield with the CNN-LSTM model (model_tsa).
    # The model outputs a scaled prediction.
    predicted_yield_scaled = model_tsa.predict(user_input_reshaped)

    # Inverse transform to get the actual predicted yield using scaler_y.
    predicted_yield = scaler_y.inverse_transform(
        predicted_yield_scaled.reshape(-1, 1)
    )[0][0]

    # Add the predicted yield to the list
    cnn_lstm_yields.append(predicted_yield)

# Append the predicted yields to the original DataFrame as a new column
df["cnn_lstm_yield"] = cnn_lstm_yields

Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/ste

In [ ]:
df.head()

,District,Crop,Year,Season,Area,Temperature,Rainfall,Yield,cnn_lstm_yield
0,1,2,1997,2,40800.0,23.998425,112.450,0.455882,0.753455
1,1,4,1997,2,900.0,23.998425,112.450,1.222222,0.752499
2,1,4,1997,3,598400.0,21.448471,9.375,0.362634,1.107845
3,1,5,1997,2,4400.0,23.998425,112.450,1.068182,0.748192
4,1,5,1997,3,6200.0,21.448471,9.375,1.467742,0.927542


In [ ]:
# Save the updated DataFrame to CSV, ready for download
output_file = "cnn_lstm_predictions.csv"
df.to_csv(output_file, index=False)
print(f"Predictions complete. The updated dataset is saved as '{output_file}'.")

Predictions complete. The updated dataset is saved as 'cnn_lstm_predictions.csv'.


## DBN model (Done locally on PC with GPU)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pgmpy.models import DynamicBayesianNetwork
# from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, MaximumLikelihoodEstimator
# from pgmpy.estimators import BicScore
from pgmpy.inference import DBNInference
from pgmpy.factors.discrete import TabularCPD
from sklearn.preprocessing import KBinsDiscretizer
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from tqdm import tqdm
import warnings
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
from tabulate import tabulate
import statsmodels.api as sm
from networkx.drawing.nx_pydot import graphviz_layout

warnings.filterwarnings("ignore")

if not hasattr(np, "product"):
    np.product = np.prod

if not hasattr(np, "float_"):
    np.float_ = np.float64

In [ ]:
def create_time_sequence(df):
    season_map = {1: "Autumn", 2: "Kharif", 3: "Rabi", 4: "Summer"}
    df["TimeStep"] = df["Year"].astype(str) + "_" + df["Season"].map(season_map)
    season_order = {1: 0, 2: 1, 3: 2, 4: 3}
    df["TimeIndex"] = ((df["Year"] - df["Year"].min()) * 4 +
                       df["Season"].apply(lambda x: season_order[x]))
    return df.sort_values("TimeIndex")

def discretize_data(df, continuous_cols, n_bins=5):
    df_disc = df.copy()
    discretizers = {}
    for col in continuous_cols:
        discretizer = KBinsDiscretizer(n_bins=n_bins, encode="ordinal",
                                       strategy="quantile")
        df_disc[f"{col}_disc"] = discretizer.fit_transform(df[[col]]).astype(int)
        discretizers[col] = discretizer
    return df_disc, discretizers

def convert_columns_str_to_tuple(df):
    new_columns = {}
    for col in df.columns:
        if isinstance(col, str) and "_" in col:
            parts = col.rsplit("_", 1)
            if len(parts) == 2 and parts[1].isdigit():
                new_columns[col] = (parts[0], int(parts[1]))
            else:
                new_columns[col] = col
        else:
            new_columns[col] = col
    return df.rename(columns=new_columns)

def change_time(node_str, new_time):
    parts = node_str.rsplit("_", 1)
    return f"{parts[0]}_{new_time}"

def str_to_tuple(node_str):
    parts = node_str.rsplit("_", 1)
    return (parts[0], int(parts[1]))

In [ ]:
def prepare_transition_data(df_remap, all_vars, cutoff_year):
    """
    Groups the remapped DataFrame by 'District' and 'Crop' to form consecutive time‑slice pairs.
    """
    train_data = df_remap[df_remap["Year_cat"] < cutoff_year].copy()
    dbn_train_data = []
    grouped = train_data.groupby(["District", "Crop"])
    for _, group in tqdm(grouped, desc="Preparing transitions"):
        group = group.sort_values("TimeIndex")
        for i in range(len(group) - 1):
            current = group.iloc[i]
            next_slice = group.iloc[i + 1]
            if next_slice["TimeIndex"] == current["TimeIndex"] + 1:
                row = {}
                for var in all_vars:
                    row[f"{var}_0"] = int(current[var])
                    row[f"{var}_1"] = int(next_slice[var])
                dbn_train_data.append(row)
    dbn_train_df = pd.DataFrame(dbn_train_data)
    return dbn_train_df

In [ ]:
def learn_dbn_structure_auto(dbn_train_df, all_vars):
    slice0_cols = [f"{var}_0" for var in all_vars]
    print("Learning intra-slice structure (time slice 0)...")
    hc_intra = HillClimbSearch(dbn_train_df[slice0_cols])
    static_model = hc_intra.estimate(scoring_method="bicscore")
    learned_intra_edges = list(static_model.edges())
    print("Learned intra-slice edges (time slice 0):")
    print(learned_intra_edges)

    intra_edges_t1 = [(change_time(u, 1), change_time(v, 1))
                      for u, v in learned_intra_edges]

    all_cols = slice0_cols + [f"{var}_1" for var in all_vars]
    disallowed_edges = []
    for s in [0, 1]:
        for t in [0, 1]:
            if not (s == 0 and t == 1):
                for a in all_vars:
                    for b in all_vars:
                        disallowed_edges.append((f"{a}_{s}", f"{b}_{t}"))
    print(f"Total disallowed edges for inter-slice learning: {len(disallowed_edges)}")
    print("Learning inter-slice edges (t0 -> t1 only)...")
    hc_inter = HillClimbSearch(dbn_train_df[all_cols])
    inter_model = hc_inter.estimate(scoring_method="bicscore",
                                    black_list=disallowed_edges)
    learned_inter_edges = [edge for edge in inter_model.edges()
                           if edge[0].endswith("_0") and edge[1].endswith("_1")]
    print("Learned inter-slice edges:")
    print(learned_inter_edges)
    return learned_intra_edges, intra_edges_t1, learned_inter_edges

def build_dbn_from_learned_structure(learned_intra_edges, intra_edges_t1, learned_inter_edges):
    dbn = DynamicBayesianNetwork()
    intra_edges_t0_tuples = [ (str_to_tuple(u), str_to_tuple(v))
                             for u, v in learned_intra_edges ]
    intra_edges_t1_tuples = [ (str_to_tuple(u), str_to_tuple(v))
                             for u, v in intra_edges_t1 ]
    inter_edges_tuples = [ (str_to_tuple(u), str_to_tuple(v))
                          for u, v in learned_inter_edges ]
    all_edges = intra_edges_t0_tuples + intra_edges_t1_tuples + inter_edges_tuples
    print("Building DBN using the learned edges...")
    for edge in all_edges:
        try:
            dbn.add_edge(edge[0], edge[1])
        except Exception as e:
            print(f"Error adding edge {edge}: {e}")
    print(f"DBN built with {len(dbn.edges())} edges.")
    return dbn

In [ ]:
def visualize_dbn_enhanced(dbn, title="Dynamic Bayesian Network", save_path=None):
    plt.figure(figsize=(15, 10))
    var_order = {
        "Year_cat": 8,
        "District": 7,
        "Crop": 6,
        "Season": 5,
        "Area_disc": 4,
        "Temperature_disc": 3,
        "Rainfall_disc": 2,
        "Yield_disc": 1,
    }
    pos = {}
    for node in dbn.nodes():
        var, t = node
        x = 2 if t == 0 else 7
        y = var_order.get(var, 0) * 1.5
        pos[node] = (x, y)
    node_colors = []
    for node in dbn.nodes():
        var = node[0]
        if var == "Yield_disc":
            node_colors.append("#ff9999")
        elif var in ["Area_disc", "Temperature_disc", "Rainfall_disc"]:
            node_colors.append("#99ccff")
        else:
            node_colors.append("#ccffcc")
    nx.draw_networkx_nodes(dbn, pos, node_size=2500, node_color=node_colors,
                           edgecolors="black", linewidths=1.5)
    labels = {node: f"{node[0]}\n(t{node[1]})" for node in dbn.nodes()}
    nx.draw_networkx_labels(dbn, pos, labels, font_size=11, font_weight="bold")
    for edge in dbn.edges():
        src, dst = edge
        style = "solid" if src[1] == dst[1] else "dashed"
        color = "#0066cc" if src[1] == dst[1] else "#cc3300"
        nx.draw_networkx_edges(dbn, pos, edgelist=[edge], arrows=True,
                               arrowstyle="-|>", arrowsize=20,
                               edge_color=color, style=style, width=2, alpha=0.8)
    ax = plt.gca()
    ax.add_patch(plt.Rectangle((0, -1), 4, 14, color="#eeeeee", alpha=0.5, zorder=-100))
    ax.add_patch(plt.Rectangle((5, -1), 4, 14, color="#f5f5f5", alpha=0.5, zorder=-100))
    plt.text(2, 13, "Time Slice 0", fontsize=14, ha="center", weight="bold")
    plt.text(7, 13, "Time Slice 1", fontsize=14, ha="center", weight="bold")

    node_legend_elements = [
        mpatches.Patch(color="#ff9999", label="Target (Yield)"),
        mpatches.Patch(color="#99ccff", label="Continuous Variables"),
        mpatches.Patch(color="#ccffcc", label="Categorical Variables"),
    ]
    edge_legend_elements = [
        mpatches.Patch(color="#0066cc", label="Intra-slice Dependency"),
        mpatches.Patch(color="#cc3300", label="Inter-slice Dependency"),
    ]
    plt.legend(handles=node_legend_elements + edge_legend_elements,
               loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=3, fontsize=12)
    plt.title(title, fontsize=18, weight="bold", pad=20)
    plt.axis("off")
    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return pos

def visualize_dbn_graphviz(dbn, title="DBN - Graphviz Hierarchical Layout", save_path=None):
    G = nx.DiGraph()
    for edge in dbn.edges():
        src, dst = edge
        src_label = f"{src[0]}_T{src[1]}"
        dst_label = f"{dst[0]}_T{dst[1]}"
        edge_style = "solid" if src[1] == dst[1] else "dashed"
        color = "#0066cc" if src[1] == dst[1] else "#cc3300"
        G.add_edge(src_label, dst_label, color=color, style=edge_style, weight=2)
    pos = graphviz_layout(G, prog="dot")
    plt.figure(figsize=(15, 12))
    node_colors = []
    for node in G.nodes():
        var = node.split("_T")[0]
        if var == "Yield_disc":
            node_colors.append("#ff9999")
        elif var in ["Area_disc", "Temperature_disc", "Rainfall_disc"]:
            node_colors.append("#99ccff")
        else:
            node_colors.append("#ccffcc")
    nx.draw_networkx_nodes(G, pos, node_size=3000, node_color=node_colors,
                           edgecolors="black", linewidths=1.5, alpha=0.8)
    edge_colors = [G[u][v]["color"] for u, v in G.edges()]
    for (u, v), color in zip(G.edges(), edge_colors):
        style = "dashed" if G[u][v]["style"] == "dashed" else "solid"
        nx.draw_networkx_edges(G, pos, edgelist=[(u, v)], width=2.0,
                               edge_color=[color], style=style,
                               arrows=True, arrowsize=20, arrowstyle='-|>', alpha=0.8)
    nx.draw_networkx_labels(G, pos, font_size=11, font_weight="bold")
    plt.legend(handles=[
        mpatches.Patch(color="#ff9999", label="Target (Yield)"),
        mpatches.Patch(color="#99ccff", label="Continuous Variables"),
        mpatches.Patch(color="#ccffcc", label="Categorical Variables"),
        mpatches.Patch(color="#0066cc", label="Intra-slice Dependency"),
        mpatches.Patch(color="#cc3300", label="Inter-slice Dependency")
    ], loc="upper center", bbox_to_anchor=(0.5, -0.05), ncol=3, fontsize=12)
    plt.title(title, fontsize=18, weight="bold")
    plt.axis("off")
    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path, dpi=300)
    plt.show()
    return pos

In [ ]:
def visualize_cpd(cpd, remap_dicts):
    if isinstance(cpd.variable, tuple):
        var_name, time_slice = cpd.variable
    else:
        parts = str(cpd.variable).rsplit('_', 1)
        if len(parts) == 2 and parts[1].isdigit():
            var_name, time_slice = parts[0], int(parts[1])
        else:
            var_name, time_slice = cpd.variable, "Unknown"

    plt.figure(figsize=(12, 8))
    if hasattr(cpd, 'evidence') and cpd.evidence:
        title = (f"CPD for {var_name} (t={time_slice}) | Conditions: "
                 f"{', '.join(str(e) for e in cpd.evidence)}")
    else:
        title = f"Prior for {var_name} (t={time_slice})"
    values = cpd.get_values()
    if values.ndim == 1:
        plt.bar(range(len(values)), values, color='skyblue', edgecolor='black')
        plt.xticks(range(len(values)), range(len(values)))
        plt.xlabel(f"States of {var_name}")
        plt.ylabel("Probability")
        if var_name in remap_dicts:
            state_info = "\nState mapping:\n"
            for i in range(len(values)):
                actual_val = remap_dicts[var_name]["reverse"].get(i, i)
                state_info += f"State {i} → {actual_val}\n"
            plt.figtext(0.02, 0.02, state_info, fontsize=10, ha='left')
    else:
        if values.ndim == 2:
            plt.imshow(values, cmap='viridis', aspect='auto')
            plt.colorbar(label='Probability')
            if hasattr(cpd, 'evidence') and len(cpd.evidence) == 1:
                plt.xlabel(f"States of {cpd.evidence[0]}")
            else:
                plt.xlabel("Parent State Combination")
            plt.ylabel(f"States of {var_name}")
        else:
            rows = cpd.variable_card
            cols = int(np.prod(values.shape) // rows)
            reshaped_values = values.reshape(rows, cols)
            plt.imshow(reshaped_values, cmap='viridis', aspect='auto')
            plt.colorbar(label='Probability')
            plt.xlabel("Parent State Combinations")
            plt.ylabel(f"States of {var_name}")
            evidence_str = ", ".join(str(e) for e in cpd.evidence)
            plt.figtext(0.5, 0.01,
                        (f"Note: This is a {values.ndim}D distribution reshaped to 2D.\n"
                         f"Parent variables: {evidence_str}\n"
                         f"Parent cardinalities: {cpd.evidence_card}"),
                        fontsize=10, ha='center')
    plt.title(title, fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
def build_continuous_bn(df):
    BN_structure = {
        "District": [],
        "Season": [],
        "Year": [],
        "Crop": [],
        "Area": [],
        "Temperature": ["District", "Season"],
        "Rainfall": ["District", "Season"],
        "Yield": ["District", "Season", "Crop", "Area", "Year", "Temperature", "Rainfall"],
    }
    cpds_BN = {}
    for node, parents in BN_structure.items():
        if len(parents) == 0:
            mu = df[node].mean()
            var = df[node].var()
            cpds_BN[node] = {"mean": mu, "variance": var}
        else:
            if node == "Yield":
                formula = "Yield ~ District + Season + Area + Year + Temperature + Rainfall + C(Crop)"
                model_fitted = sm.OLS.from_formula(formula, data=df).fit()
                cpds_BN[node] = {"model": model_fitted, "params": model_fitted.params,
                                 "sigma": np.sqrt(model_fitted.mse_resid)}
            else:
                X = df[parents]
                X = sm.add_constant(X)
                y = df[node]
                model_fitted = sm.OLS(y, X).fit()
                cpds_BN[node] = {"model": model_fitted, "params": model_fitted.params,
                                 "sigma": np.sqrt(model_fitted.mse_resid)}
    return BN_structure, cpds_BN

def predict_yield_bn(cpds_BN, test_input):
    crop_mapping = {
        1: "Cotton", 2: "Gram", 3: "Groundnut", 4: "Jowar", 5: "Maize",
        6: "Moong", 7: "Mustard", 8: "Rice", 9: "Sesamum", 10: "Small Millets",
        11: "Soyabean", 12: "Sugarcane", 13: "Sunflower", 14: "Tur", 15: "Urad"
    }
    yield_model = cpds_BN["Yield"]["model"]
    predictions = {}
    print("\n=== BN Predicted Yields for Each Crop ===\n")
    for crop_code in range(1, 16):
        test_df = pd.DataFrame({
            "District": [test_input["District"]],
            "Season": [test_input["Season"]],
            "Crop": [crop_code],
            "Area": [test_input["Area"]],
            "Year": [test_input["Year"]],
            "Temperature": [test_input["Temperature"]],
            "Rainfall": [test_input["Rainfall"]]
        })
        predicted_yield = yield_model.predict(test_df)[0]
        predictions[crop_code] = predicted_yield
        print(f"Crop: {crop_mapping[crop_code]} (encoded {crop_code}): Predicted Yield = {predicted_yield:.4f}")
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    print("\n=== Crops Arranged by Descending Predicted Yield ===")
    for crop_code, pred in sorted_predictions:
        print(f"{crop_mapping[crop_code]}: {pred:.4f}")
    return predictions

In [ ]:
df = pd.read_csv("final_df.csv")
print("Data loaded. Shape:", df.shape)

df_time = create_time_sequence(df)
print("Time sequence created.")

continuous_vars = ["Area", "Temperature", "Rainfall", "Yield"]
df_disc, discretizers = discretize_data(df_time, continuous_vars, n_bins=5)
print("Continuous variables discretized.")

cols_to_convert = ["District", "Crop", "Season", "Year"] + [f"{var}_disc" for var in continuous_vars]
for col in cols_to_convert:
    df_disc[col] = df_disc[col].astype(int)

base_year = df_disc["Year"].min()
df_disc["Year_diff"] = df_disc["Year"] - base_year
bins = [0, 5, 10, 15, np.inf]
df_disc["Year_cat"] = pd.cut(df_disc["Year_diff"], bins=bins, labels=[0, 1, 2, 3],
                             right=False).astype(int)

all_vars = ["District", "Crop", "Season", "Year_cat", "Area_disc",
            "Temperature_disc", "Rainfall_disc", "Yield_disc"]

df_remap = df_disc.copy()
remap_dicts = {}
for var in all_vars:
    unique_vals = sorted(df_remap[var].unique())
    remap = {val: idx for idx, val in enumerate(unique_vals)}
    reverse_map = {idx: val for val, idx in remap.items()}
    df_remap[var] = df_remap[var].map(remap)
    remap_dicts[var] = {"forward": remap, "reverse": reverse_map}
    print(f"{var} remapped: {len(unique_vals)} states")

all_years_cat = sorted(df_remap["Year_cat"].unique())
train_years_count = int(len(all_years_cat) * 0.8)
cutoff_year_cat = all_years_cat[train_years_count]
print(f"Training data: Year_cat < {cutoff_year_cat}")

Data loaded. Shape: (13904, 8)
Time sequence created.
Continuous variables discretized.
District remapped: 32 states
Crop remapped: 15 states
Season remapped: 4 states
Year_cat remapped: 4 states
Area_disc remapped: 5 states
Temperature_disc remapped: 5 states
Rainfall_disc remapped: 5 states
Yield_disc remapped: 5 states
Training data: Year_cat < 3


In [ ]:
from tqdm import tqdm

In [ ]:
dbn_train_df = prepare_transition_data(df_remap, all_vars, cutoff_year_cat)
print(f"Number of training transitions: {len(dbn_train_df)}")

learned_intra_edges, intra_edges_t1, learned_inter_edges = learn_dbn_structure_auto(dbn_train_df, all_vars)

dbn_auto = build_dbn_from_learned_structure(learned_intra_edges, intra_edges_t1,
                                              learned_inter_edges)

dbn_train_df = convert_columns_str_to_tuple(dbn_train_df)

def convert_dbn_to_bayesian_model_str(dbn):
    edges = []
    for u, v in dbn.edges():
        u_str = f"{u[0]}_{u[1]}"
        v_str = f"{v[0]}_{v[1]}"
        edges.append((u_str, v_str))
    bayes_model = BayesianModel(edges)
    return bayes_model

def convert_tuple_columns_to_str(df):
    new_df = df.copy()
    new_df.columns = [
        f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col
        for col in new_df.columns
    ]
    return new_df

bayes_model_str = convert_dbn_to_bayesian_model_str(dbn_auto)

dbn_train_df_str = convert_tuple_columns_to_str(dbn_train_df)

print("Bayesian model nodes:", bayes_model_str.nodes())
print("DataFrame columns:", dbn_train_df_str.columns.tolist())

print("\nLearning CPDs using Maximum Likelihood Estimation on the converted BayesianModel...")
bayes_model_str.fit(dbn_train_df_str, estimator=MaximumLikelihoodEstimator)
print("Discrete BN CPD Learning complete (via BayesianModel conversion).")

cpds_list = bayes_model_str.get_cpds()
print(f"\nNumber of CPDs in the model: {len(cpds_list)}")

if len(cpds_list) == 0:
    print("No CPDs were learned!")
else:
    print("\n==== DISCRETE BN CPDs ====")
    for cpd in cpds_list:
        print(f"\nCPD for {cpd.variable}:")
        if hasattr(cpd, 'evidence') and cpd.evidence:
            evidence_str = ", ".join(str(e) for e in cpd.evidence)
            print(f"Conditioned on: {evidence_str}")
            print(f"Evidence cardinalities: {cpd.evidence_card}")
        else:
            print("This is a prior probability")

        print(f"Variable cardinality: {cpd.variable_card}")
        print("Values shape:", cpd.values.shape)

        if cpd.values.ndim <= 2:
            try:
                from tabulate import tabulate
                print(tabulate(pd.DataFrame(cpd.values), headers="keys", tablefmt="psql"))
            except Exception as e:
                print("Error printing CPD table:", e)
        else:
            print(f"Mean probability: {cpd.values.mean():.4f}")
            print(f"Min probability: {cpd.values.min():.4f}")
            print(f"Max probability: {cpd.values.max():.4f}")

Preparing transitions: 100%|██████████| 439/439 [00:00<00:00, 573.14it/s]

Number of training transitions: 1598
Learning intra-slice structure (time slice 0)...


ValueError: The scoring method names have been changed. Please refer the documentation.

# Weighted Linear Integration

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/final_df_with_dbn+cnn-lstm_yield.csv')

In [ ]:
df.head()

,CNN_LSTM_Yield,DBN_Yield,Yield
0,0.753455,0.9,0.455882
1,0.752499,1.3,1.222222
2,1.107845,0.5,0.362634
3,0.748192,1.5,1.068182
4,0.927542,1.9,1.467742


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

filepath = "final_df_with_dbn+cnn-lstm_yield.csv"
df = pd.read_csv(filepath)

X = df[['CNN_LSTM_Yield', 'DBN_Yield']]
y = df['Yield']

model = LinearRegression()
model.fit(X, y)

intercept = model.intercept_
coefficients = model.coef_

print("Intercept (bias):", intercept)
print("Coefficients (weights):", coefficients)

equation = (f"Yield = {intercept:.4f} + "
            f"{coefficients[0]:.4f} * (CNN_LSTM_Yield) + "
            f"{coefficients[1]:.4f} * (DBN_Yield)")
print("Regression Equation:")
print(equation)

Intercept (bias): 0.06208483664047437
Coefficients (weights): [ 1.58515847 -0.65674222]
Regression Equation:
Yield = 0.0621 + 1.5852 * (CNN_LSTM_Yield) + -0.6567 * (DBN_Yield)
